In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import time
from pathlib import Path
import json
import logging
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from contextlib import contextmanager

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class DatabaseConfig:
    """Configuration for database setup"""
    db_path: Path
    enable_wal_mode: bool = True
    enable_foreign_keys: bool = True
    cache_size_mb: int = 256
    temp_store: str = "MEMORY"
    synchronous: str = "NORMAL"
    journal_mode: str = "WAL"

class VinylCatalogDB:
    """High-performance database interface for vinyl catalog"""
    
    def __init__(self, config: DatabaseConfig):
        self.config = config
        self.db_path = config.db_path
        self._ensure_db_directory()
        
    def _ensure_db_directory(self):
        """Ensure database directory exists"""
        self.db_path.parent.mkdir(parents=True, exist_ok=True)
        
    @contextmanager
    def get_connection(self):
        """Context manager for database connections with optimizations"""
        conn = sqlite3.connect(str(self.db_path))
        
        # Apply performance optimizations
        conn.execute(f"PRAGMA cache_size = -{self.config.cache_size_mb * 1024}")
        conn.execute(f"PRAGMA temp_store = {self.config.temp_store}")
        conn.execute(f"PRAGMA synchronous = {self.config.synchronous}")
        conn.execute(f"PRAGMA journal_mode = {self.config.journal_mode}")
        
        if self.config.enable_foreign_keys:
            conn.execute("PRAGMA foreign_keys = ON")
            
        # Enable row factory for dict-like access
        conn.row_factory = sqlite3.Row
        
        try:
            yield conn
        finally:
            conn.close()
            
    def create_schema(self):
        """Create optimized database schema"""
        with self.get_connection() as conn:
            # Main releases table with core data
            conn.execute("""
                CREATE TABLE IF NOT EXISTS releases (
                    release_id INTEGER PRIMARY KEY,
                    discogs_id INTEGER UNIQUE,
                    title TEXT NOT NULL,
                    artist TEXT NOT NULL,
                    album TEXT,
                    year INTEGER,
                    genre TEXT,
                    label TEXT,
                    country TEXT,
                    format TEXT,
                    status TEXT,
                    catalog_number TEXT,
                    master_id INTEGER,
                    
                    -- Quality and popularity metrics
                    popularity_score REAL DEFAULT 0.0,
                    rating REAL DEFAULT 0.0,
                    plays INTEGER DEFAULT 0,
                    favorites INTEGER DEFAULT 0,
                    
                    -- Metadata
                    duration INTEGER, -- in seconds
                    data_quality TEXT,
                    source TEXT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            """)
            
            # Artists table for normalized artist data
            conn.execute("""
                CREATE TABLE IF NOT EXISTS artists (
                    artist_id INTEGER PRIMARY KEY,
                    discogs_artist_id INTEGER UNIQUE,
                    name TEXT NOT NULL,
                    real_name TEXT,
                    profile TEXT,
                    urls TEXT,
                    name_variations TEXT,
                    aliases TEXT,
                    quality_score INTEGER DEFAULT 0,
                    image_count INTEGER DEFAULT 0,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            """)
            
            # Labels table
            conn.execute("""
                CREATE TABLE IF NOT EXISTS labels (
                    label_id INTEGER PRIMARY KEY,
                    discogs_label_id INTEGER UNIQUE,
                    name TEXT NOT NULL,
                    contact_info TEXT,
                    parent_label_id INTEGER,
                    profile TEXT,
                    data_quality TEXT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (parent_label_id) REFERENCES labels(label_id)
                )
            """)
            
            # Masters table for release grouping
            conn.execute("""
                CREATE TABLE IF NOT EXISTS masters (
                    master_id INTEGER PRIMARY KEY,
                    discogs_master_id INTEGER UNIQUE,
                    title TEXT NOT NULL,
                    original_year INTEGER,
                    main_release_id INTEGER,
                    num_versions INTEGER DEFAULT 1,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            """)
            
            # Genres table for normalization
            conn.execute("""
                CREATE TABLE IF NOT EXISTS genres (
                    genre_id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT UNIQUE NOT NULL,
                    parent_genre_id INTEGER,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (parent_genre_id) REFERENCES genres(genre_id)
                )
            """)
            
            # Release-Artist junction table (many-to-many)
            conn.execute("""
                CREATE TABLE IF NOT EXISTS release_artists (
                    release_id INTEGER,
                    artist_id INTEGER,
                    role TEXT DEFAULT 'artist',
                    join_phrase TEXT,
                    anv TEXT, -- Artist Name Variation
                    PRIMARY KEY (release_id, artist_id, role),
                    FOREIGN KEY (release_id) REFERENCES releases(release_id),
                    FOREIGN KEY (artist_id) REFERENCES artists(artist_id)
                )
            """)
            
            # Tracklist table
            conn.execute("""
                CREATE TABLE IF NOT EXISTS tracklist (
                    track_id INTEGER PRIMARY KEY AUTOINCREMENT,
                    release_id INTEGER NOT NULL,
                    position TEXT NOT NULL,
                    title TEXT NOT NULL,
                    duration INTEGER, -- in seconds
                    FOREIGN KEY (release_id) REFERENCES releases(release_id)
                )
            """)
            
            # Reviews table for future MARD integration
            conn.execute("""
                CREATE TABLE IF NOT EXISTS reviews (
                    review_id INTEGER PRIMARY KEY AUTOINCREMENT,
                    release_id INTEGER NOT NULL,
                    reviewer TEXT,
                    rating REAL,
                    review_text TEXT,
                    sentiment_score REAL,
                    source TEXT,
                    review_date DATE,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (release_id) REFERENCES releases(release_id)
                )
            """)
            
            # User interactions for recommendation system
            conn.execute("""
                CREATE TABLE IF NOT EXISTS user_interactions (
                    interaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id TEXT NOT NULL,
                    release_id INTEGER NOT NULL,
                    interaction_type TEXT NOT NULL, -- 'play', 'favorite', 'skip', 'rate'
                    value REAL, -- rating value, play count, etc.
                    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (release_id) REFERENCES releases(release_id)
                )
            """)
            
            conn.commit()
            logger.info("✅ Database schema created successfully")
    
    def create_indexes(self):
        """Create performance indexes"""
        indexes = [
            # Primary search indexes
            "CREATE INDEX IF NOT EXISTS idx_releases_artist ON releases(artist)",
            "CREATE INDEX IF NOT EXISTS idx_releases_title ON releases(title)",
            "CREATE INDEX IF NOT EXISTS idx_releases_album ON releases(album)",
            "CREATE INDEX IF NOT EXISTS idx_releases_genre ON releases(genre)",
            "CREATE INDEX IF NOT EXISTS idx_releases_year ON releases(year)",
            "CREATE INDEX IF NOT EXISTS idx_releases_label ON releases(label)",
            
            # Composite indexes for common queries
            "CREATE INDEX IF NOT EXISTS idx_releases_artist_year ON releases(artist, year)",
            "CREATE INDEX IF NOT EXISTS idx_releases_genre_year ON releases(genre, year)",
            "CREATE INDEX IF NOT EXISTS idx_releases_popularity ON releases(popularity_score DESC)",
            "CREATE INDEX IF NOT EXISTS idx_releases_rating ON releases(rating DESC)",
            
            # Foreign key indexes
            "CREATE INDEX IF NOT EXISTS idx_release_artists_release ON release_artists(release_id)",
            "CREATE INDEX IF NOT EXISTS idx_release_artists_artist ON release_artists(artist_id)",
            "CREATE INDEX IF NOT EXISTS idx_tracklist_release ON tracklist(release_id)",
            "CREATE INDEX IF NOT EXISTS idx_reviews_release ON reviews(release_id)",
            "CREATE INDEX IF NOT EXISTS idx_user_interactions_user ON user_interactions(user_id)",
            "CREATE INDEX IF NOT EXISTS idx_user_interactions_release ON user_interactions(release_id)",
            
            # Search optimization indexes
            "CREATE INDEX IF NOT EXISTS idx_artists_name ON artists(name)",
            "CREATE INDEX IF NOT EXISTS idx_labels_name ON labels(name)",
            "CREATE INDEX IF NOT EXISTS idx_masters_title ON masters(title)",
            
            # Full-text search preparation
            "CREATE INDEX IF NOT EXISTS idx_releases_search_terms ON releases(artist, title, album, genre, label)",
        ]
        
        with self.get_connection() as conn:
            for index_sql in indexes:
                try:
                    conn.execute(index_sql)
                    logger.info(f"Created index: {index_sql.split('idx_')[1].split(' ')[0]}")
                except sqlite3.Error as e:
                    logger.warning(f"Failed to create index: {e}")
            
            conn.commit()
            logger.info("✅ All indexes created successfully")
    
    def analyze_database(self):
        """Update SQLite statistics for query optimization"""
        with self.get_connection() as conn:
            conn.execute("ANALYZE")
            conn.commit()
            logger.info("✅ Database statistics updated")

# Setup paths and configuration
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR = PROJECT_ROOT / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
DB_DIR = DATA_DIR / 'database'

# Initialize database
db_config = DatabaseConfig(
    db_path=DB_DIR / 'vinyl_catalog.db',
    cache_size_mb=512,  # Use 512MB cache for better performance
)

vinyl_db = VinylCatalogDB(db_config)

print(f"🗄️ Database will be created at: {vinyl_db.db_path}")
print(f"📁 Project structure:")
print(f"  - Data directory: {DATA_DIR}")
print(f"  - Processed directory: {PROCESSED_DIR}")
print(f"  - Database directory: {DB_DIR}")